In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import math
import random
from collections import deque

#import random

In [ ]:
"""class DeepQNetwork(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
        super(DeepQNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions

        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(*self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(*self.fc2_dims, self.n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)

        return actions
"""


"class DeepQNetwork(nn.Module):\n    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):\n        super(DeepQNetwork, self).__init__()\n        self.input_dims = input_dims\n        self.fc1_dims = fc1_dims\n        self.fc2_dims = fc2_dims\n        self.n_actions = n_actions\n\n        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)\n        self.fc2 = nn.Linear(*self.fc1_dims, self.fc2_dims)\n        self.fc3 = nn.Linear(*self.fc2_dims, self.n_actions)\n        self.optimizer = optim.Adam(self.parameters(), lr=lr)\n        self.loss = nn.MSELoss()\n        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')\n        self.to(self.device)\n\n    def forward(self, state):\n        x = F.relu(self.fc1(state))\n        x = F.relu(self.fc2(x))\n        actions = self.fc3(x)\n\n        return actions\n"

In [ ]:
class Agent():
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # n previous days
        self.action_size = 3 # hold, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval 
        self.gamma = 0.99
        self.epsilon = 1.0 
        self.epsilon_min = 0.01
        self.epsilon_decay = (0.995)
        self.model = load_model(model_name) if is_eval else self._model()
        
    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        #model.add(Dense(units=32, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size , activation="linear"))
        
        opt = Adam(learning_rate=0.001)
        model.compile(loss="mse")#  optimizer = opt) #optimizer=keras.optimizers.adam_v2(lr=0.001))
        return model

    def act(self, state):
        """if not self.is_eval and random.random()<= self.epsilon:#exploration
            return random.randrange(self.action_size)
        options = self.model.predict(state)
        return np.argmax(options[0])"""

        if not self.is_eval and random.random()<= self.epsilon:#exploration
            return random.randrange(self.action_size)
        else :#exploitation
            options = self.model.predict(state)
            return np.argmax(options[0])
    
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])#Bellman Eq
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

            


In [ ]:
def formatPrice(n):
    return("-Rs." if n<0 else "Rs.")+"{0:.2f}".format(abs(n))
    
def getStockDataVec(key):
    vec = []
    lines = open(key+".csv","r").read().splitlines()
    for line in lines[1:]:
        #print(line)
        #print(float(line.split(",")[4]))
        vec.append(float(line.split(",")[4]))
        #print(vec)
    return vec 

def getState(data, t, n):
    d = t-n+1

    window = data[d:t+1] if d>=0 else -d*[data[0]] + data[0:t+1] #pad with time 0
    
    #sigmoid variation avoid
    return np.array([window])
        

In [ ]:
# Configuration paramaters for the whole setup
# seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 1  # Size of batch taken from replay buffer
max_steps_per_episode = 10000


Max_Transactions = 100#maximum number of time we can do a buy after a selling
Total_Money =  100000  #total money that we have in cash(virtual ) 
bought_price = 0
Current_Stocks_Bought = 0# the number of stocks we have bought currently
Current_Transaction_Count = 0 

In [ ]:
import sys
max_episodes = 10
window_size = 15 # n sized window gives state
agent = Agent(window_size)
stock_name = "CSCO"
data = getStockDataVec(stock_name)
t_max = len(data) 
print(t_max)

original_stdout = sys.stdout
OutputFile = open("Output1.txt", "w")

sys.stdout = OutputFile
for episode in range(max_episodes):

    #OutputFile.write("Episode " + str(episode) + "/" + str(max_episodes))
    print("Episode " + str(episode) + "/" + str(max_episodes))
    state = getState(data, 0, window_size)
    total_profit = 0

    for time in range(t_max-1):
        action = agent.act(state)

        next_state = getState(data, time+1, window_size)
        reward = 0 #Hold: action == 0
        if action == 0:
           reward = (data[time] - data[time-1])* Current_Stocks_Bought
           print("time: "+ str(time) +" Hold: " + formatPrice(data[time]))

        if action == 1 and Current_Transaction_Count < Max_Transactions:#buy
            x = Total_Money/(Max_Transactions - Current_Transaction_Count)
            Total_Money -= x
            bought_price += x
            Current_Stocks_Bought += x/data[time]
            Current_Transaction_Count += 1
            #OutputFile.write("time: "+ str(time) +" Buy: " + formatPrice(data[time]))
            print("time: "+ str(time) +" Buy: " + formatPrice(data[time]))

        elif action == 2: #sell
            sell_price = data[time] * Current_Stocks_Bought
            reward = sell_price - bought_price
            total_profit += reward

            Total_Money += sell_price
            bought_price = 0
            Current_Stocks_Bought = 0
            Current_Transaction_Count = 0
            #OutputFile.write("time: "+str(time) +" Sell: " + formatPrice(data[time]) + " | Profit: " + formatPrice(sell_price - bought_price)+ " | Total Profit: " + formatPrice(total_profit))
            print("time: "+str(time) +" Sell: " + formatPrice(data[time]) + " | Profit: " + formatPrice(sell_price - bought_price)
            + " | Total Profit: " + formatPrice(total_profit))


        done = True if time == t_max-2 else False
        agent.memory.append((state, action, reward, next_state, done))#tuple

        state = next_state
        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    #if episode % 10 == 0:
    agent.model.save(str(episode))

sys.stdout = original_stdout

        

            




443
INFO:tensorflow:Assets written to: 0/assets
INFO:tensorflow:Assets written to: 1/assets
INFO:tensorflow:Assets written to: 2/assets
INFO:tensorflow:Assets written to: 3/assets
INFO:tensorflow:Assets written to: 4/assets
INFO:tensorflow:Assets written to: 5/assets
INFO:tensorflow:Assets written to: 6/assets
INFO:tensorflow:Assets written to: 7/assets
INFO:tensorflow:Assets written to: 8/assets
INFO:tensorflow:Assets written to: 9/assets


In [ ]:
model_name = "9"
stock_name = "CSC"
model = load_model(model_name)
window_size = model.layers[0].input.shape.as_list()[1]
agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
print(data)
t_max = len(data) 
print(t_max)

#original_stdout = sys.stdout
#OutputFile = open("Output.txt", "w")

#sys.stdout = OutputFile


Max_Transactions = 100 #maximum number of time we can do a buy after a selling
Total_Money =  100000 #total money that we have in cash(virtual ) 
bought_price = 0
Current_Stocks_Bought = 0 # the number of stocks we have bought currently
Current_Transaction_Count = 0 

state = getState(data, 0, window_size)
total_profit = 0

for time in range(t_max-1):
    action = agent.act(state)

    next_state = getState(data, time+1, window_size)
    reward = 0 #Hold: action == 0

    if action == 0:
           reward = (data[time] - data[time-1])* Current_Stocks_Bought
           print("time: "+ str(time) +" Hold: " + formatPrice(data[time]))


    if action == 1 and Current_Transaction_Count < Max_Transactions:#buy
        x = Total_Money/(Max_Transactions - Current_Transaction_Count)
        Total_Money -= x
        bought_price += x
        Current_Stocks_Bought += x/data[time]
        Current_Transaction_Count += 1
        #OutputFile.write("time: "+ str(time) +" Buy: " + formatPrice(data[time]))
        print("time: "+ str(time) +" Buy: " + formatPrice(data[time]))

    if action == 2 or time == t_max-2: #sell
        sell_price = data[time] * Current_Stocks_Bought
        reward = sell_price - bought_price
        total_profit += reward

        Total_Money += sell_price
        bought_price = 0
        Current_Stocks_Bought = 0
        Current_Transaction_Count = 0
        #OutputFile.write("time: "+str(time) +" Sell: " + formatPrice(data[time]) + " | Profit: " + formatPrice(sell_price - bought_price)+ " | Total Profit: " + formatPrice(total_profit))
        print("time: "+str(time) +" Sell: " + formatPrice(data[time]) + " | Profit: " + formatPrice(sell_price - bought_price)
        + " | Total Profit: " + formatPrice(total_profit))


    done = True if time == t_max-2 else False
    agent.memory.append((state, action, reward, next_state, done))#tuple

    state = next_state
    if done:
        print("--------------------------------")
        print("Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")




[27.450001, 27.66, 28.02, 28.799999, 29.059999, 28.65, 27.950001, 28.15, 28.190001, 28.280001, 28.6, 28.719999, 28.379999, 27.790001, 27.559999, 27.07, 26.540001, 26.75, 26.120001, 26.129999, 25.43, 26.24, 26.24, 25.870001, 26.290001, 25.85, 25.15, 24.33, 24.299999, 23.48, 24.030001, 25.110001, 24.200001, 24.1, 24.059999, 24.360001, 24.5, 24.940001, 23.82, 23.26, 23.08, 23.379999, 23.540001, 23.52, 23.43, 24.059999, 23.530001, 23.299999, 22.879999, 23.200001, 23.190001, 23.6, 23.799999, 24.07, 24.950001, 24.66, 24.389999, 24.4, 24.290001, 24.27, 23.940001, 24.110001, 23.99, 25.15, 25.139999, 24.950001, 24.32, 24.290001, 25.58, 24.469999, 24.77, 25.639999, 25.75, 24.780001, 24.18, 24.08, 24.09, 24.98, 24.959999, 24.23, 24.389999, 23.940001, 23.959999, 23.530001, 24.040001, 23.379999, 23.309999, 23.110001, 23.83, 23.889999, 24.51, 24.889999, 24.92, 25.440001, 26.030001, 25.6, 25.35, 25.51, 25.639999, 26.67, 26.75, 26.280001, 26.33, 25.780001, 25.700001, 25.49, 25.84, 25.889999, 25.75, 26